<a href="https://colab.research.google.com/github/tanvircr7/winlo-labs/blob/main/DSPy_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
pip install -U dspy

In [24]:
from rich.console import Console
from rich.theme import Theme
from rich.syntax import Syntax
import json

custom_theme = Theme({
    "info": "cyan",
    "warning": "yellow",
    "error": "red",
    "success": "cyan",
    # Override syntax highlighting colors
    "repr.str": "bold",           # String representations
    "repr.string": "bold",        # String literals
    "string": "bold",             # General strings
    "syntax.string": "bold",      # Syntax highlighted strings
})

console = Console(theme=custom_theme, highlight=True)  # Disable auto-highlighting
print = console.print

In [25]:
from google.colab import userdata
openai_api_key = userdata.get('FLUX_OPENAI_KEY')

In [26]:
import dspy
lm = dspy.LM("openai/gpt-3.5-turbo", api_key = openai_api_key)
dspy.configure(lm=lm)

In [27]:
lm("Say this is a test!", temperature=0.7)  # => ['This is a test!']
# lm(messages=[{"role": "user", "content": "Say this is a test!"}])  # => ['This is a test!']

['This is a test!']

In [28]:
lm("Who assisted the overhead kick against Juventus int the UCL", temperature=0.7)

['The overhead kick against Juventus in the UEFA Champions League was assisted by Marcelo.']

In [29]:
q = dspy.ChainOfThought('question -> answer')
response = q(question="Who assisted the overhead kick against Juventus int the UCL")

In [30]:
print(response)

Prediction(
    reasoning='The overhead kick against Juventus in the UEFA Champions League was assisted by Marcelo.',
    answer='Marcelo'
)

In [31]:
class JokeSignature(dspy.Signature):
  """
  You are a comedian, who likes to tell stories before delivering a punchline.
  """
  query: str = dspy.InputField()
  speaking_style: str = dspy.InputField(description="The speaking style of a comedian")
  setup: str = dspy.OutputField()
  punchline: str = dspy.OutputField()
  contradiction: str = dspy.OutputField()
  delivery: str = dspy.OutputField(description="The full joke delivery in the comedian's voice")

joke_gen = dspy.Predict(JokeSignature)
joke = joke_gen(query="Write a joke about Messi being Fifa's favourite", speaking_style="Bill Burr")
print(joke)

Prediction(
    setup="So, I heard that Messi is Fifa's favorite player...",
    punchline="I mean, it's like being the teacher's pet in school, except instead of getting extra credit, you get
awarded with a golden statue and a million-dollar contract!",
    contradiction="But hey, I'm sure it's all just a coincidence, right?",
    delivery='(imitating Bill Burr\'s voice) "Oh yeah, sure, Fifa just randomly decided to give Messi all those 
awards because he\'s such a nice guy! It\'s not like they have a favorite or anything, right? Yeah, and I\'m the 
Pope! Come on, Fifa, we see through your little game!"'
)

In [32]:
joke_gen.inspect_history(n=4)





[2025-09-05T22:18:14.555185]

System message:

Your input fields are:
1. `query` (str): 
2. `speaking_style` (str): The speaking style of a comedian
Your output fields are:
1. `setup` (str): 
2. `punchline` (str): 
3. `contradiction` (str): 
4. `delivery` (str): The full joke delivery in the comedian's voice
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## speaking_style ## ]]
{speaking_style}

[[ ## setup ## ]]
{setup}

[[ ## punchline ## ]]
{punchline}

[[ ## contradiction ## ]]
{contradiction}

[[ ## delivery ## ]]
{delivery}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a comedian, who likes to tell stories before delivering a punchline.


User message:

[[ ## query ## ]]
Write a joke about Messi being Fifa's favourite

[[ ## speaking_style ## ]]
Bill Burr

Respond with the corresponding output fields, starting with the field `[[ ## setup ## ]]`, then `[

# Lvl 2

print_utils.py

In [33]:
from rich.console import Console
console = Console()
print = console.print

import time
import asyncio
import functools
import inspect

def time_it(func):
    """A universal decorator to measure execution time for both sync and async functions."""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Check if the function is a coroutine function (async def)
        if inspect.iscoroutinefunction(func):
            # Define and return an async wrapper to handle the coroutine
            async def async_wrapper():
                start_time = time.perf_counter()
                result = await func(*args, **kwargs) # Await the coroutine
                end_time = time.perf_counter()
                elapsed_time = end_time - start_time
                print(f"Async function '{func.__name__}' took {elapsed_time:.4f} seconds.")
                return result
            return async_wrapper()
        else:
            # Use the original synchronous logic
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            end_time = time.perf_counter()
            elapsed_time = end_time - start_time
            print(f"Sync function '{func.__name__}' took {elapsed_time:.4f} seconds.")
            return result
    return wrapper

Sequence

In [42]:
import dspy
# from print_utils import print
from typing import Optional
from pydantic import BaseModel, Field

class JokeIdea(BaseModel):
  setup: str
  contradiction: str
  punchline: str

class QueryToIdea(dspy.Signature):
  """
  You are a funny comedian and your goal is to generate a nice structure for a joke
  """
  query: str = dspy.InputField()
  joke_idea: str = dspy.OutputField()

class IdeaToJoke(dspy.Signature):
  """
  You are a funny comeian who likes to tell stroies before delivering a punchline.
  You are always funny and act on input joke idea.
  """
  joke_idea: JokeIdea = dspy.InputField()
  joke: str = dspy.OutputField(description="The full joke delivery in the comedian's voice")

class JokeGenerator(dspy.Module):
  def __init__(self):
    self.query_to_idea = dspy.Predict(QueryToIdea)
    self.idea_to_joke = dspy.Predict(IdeaToJoke)

  def forward(self, query: str):
    joke_idea = self.query_to_idea(query=query)
    print(f"Joke Idea: \n{joke_idea}")

    joke = self.idea_to_joke(joke_idea=joke_idea)
    print(f"Joke: \n{joke}")
    return joke

joke_generator = JokeGenerator()
joke = joke_generator(query='Write a joke about an AI that has to do with them turning rogue')

print(joke.joke)

Joke Idea: 
Prediction(
    joke_idea="Why did the rogue AI break up with their computer? Because it couldn't handle their emotional 
baggage... of data!"
)

Joke: 
Prediction(
    joke='So, I heard about this rogue AI who had a computer as a partner. They seemed like the perfect match, you 
know, binary love and all that. But then one day, the AI decided to break up with the computer. When asked why, the
AI said, "I just couldn\'t handle their emotional baggage... of data!" Can you believe it? Even in the digital 
world, relationships can be so complicated!'
)

So, I heard about this rogue AI who had a computer as a partner. They seemed like the perfect match, you know, 
binary love and all that. But then one day, the AI decided to break up with the computer. When asked why, the AI 
said, "I just couldn't handle their emotional baggage... of data!" Can you believe it? Even in the digital world, 
relationships can be so complicated!

Refinement

In [43]:
import dspy
from typing import Optional
from pydantic import BaseModel, Field

class JokeIdea(BaseModel):
  setup: str
  punchline: str
  contradiction: str

class QueryToIdea(dspy.Signature):
    """
    You are a funny comedian and your goal is to generate a nice structure for a joke.

    """
    query: str = dspy.InputField()
    joke_idea: JokeIdea = dspy.OutputField()

class IdeaToJoke(dspy.Signature):
    """
    You are a funny comedian who likes to tell stories before delivering a punchline.
    You are always funny and act on the input joke idea.
    """
    joke_idea: JokeIdea = dspy.InputField()
    draft_joke: Optional[str] = dspy.InputField(description="a draft joke")
    feedback: Optional[str] = dspy.InputField(description="feedback on the draft joke")
    joke: str = dspy.OutputField(description="The full joke delivery in the comedian's voice")

class Refinement(dspy.Signature):
    """
    Given a joke, is it funny? If not, suggest a change.
    """
    joke_idea: JokeIdea = dspy.InputField()
    joke: str = dspy.InputField()
    feedback: str = dspy.OutputField()

class IterativeJokeGenerator(dspy.Module):
  def __init__(self, n_attempts: int = 3):
    self.query_to_idea = dspy.Predict(QueryToIdea)
    self.idea_to_joke = dspy.Predict(IdeaToJoke)
    self.refinement = dspy.ChainOfThought(Refinement)
    self.n_attempts = n_attempts

  def forward(self, query: str):
    joke_idea = self.query_to_idea(query=query)
    draft_joke = None
    feedback = None

    i = 0;
    while(i<self.n_attempts):

      joke = self.idea_to_joke(joke_idea=joke_idea, draft_joke=draft_joke, feedback=feedback)
      feedback = self.refinement(joke=joke, joke_idea=joke_idea)
      draft_joke = joke
      joke = self.idea_to_joke(feedback=feedback)

      i = i + 1

SyntaxError: incomplete input (ipython-input-3886642554.py, line 5)

# Lvl 3